In [1]:
import numpy as np
import time

In [2]:
"""
System parameters
"""
n = 2000
tol = 1e-8
maxiter = 100

In [3]:
"""
Create sparse matrix
"""
sparsity = 0.01
A = np.zeros((n,n))
for i in range(0,n):
    A[i,i] = i+1
# randn is normal distribution
A += sparsity*np.random.randn(n,n)
A = (A.T+A)/2

In [4]:
"""
Define quantities needed in Davidson
"""
eigs = 3 # No. of eigenvectors to solve for
k = 2*eigs # No. of trial vectors is about double that of no. of eigs
t = np.eye(n,k) # 1000 by 8, so the columns are the trial unit vectors
V = np.zeros((n,n))
I = np.eye(n)

In [5]:
"""
The main Davidson routine
"""
start_davidson = time.time()
for m in range(k,maxiter,k):
    if m <= k:
        # if this is the first iteration
        for j in range(0,k):
            V[:,j] = t[:,j]
        theta_old = 1
    else:
        theta_old = theta[:eigs]
    # 0-m is the subspace
    # V is the orthonormalised matrix, and R is discarded
    V[:,:m], R = np.linalg.qr(V[:,:m])
    # T = V^T A V
    T = np.dot(V[:,:m].T,np.dot(A,V[:,:m]))
    t_eigval, t_eigv = np.linalg.eigh(T)
    # idx is an array of indices that would sort t_eigval from smallest to biggest
    # The BLAS dsyev already sorts them so no worries of having to write a sorting algo
    idx = t_eigval.argsort()
    theta = t_eigval[idx]
    s = t_eigv[:,idx]
    for j in range(0,k):
        # w = (A-It) V s
        w = np.dot((A-theta[j]*I),np.dot(V[:,:m],s[:,j]))
        q = w/(theta[j]-A[j,j])
        V[:,(m+j)] = q
    # Check for convergence
    norm = np.linalg.norm(theta[:eigs] - theta_old)
    if norm < tol:
        break
end_davidson = time.time()

In [6]:
print("davidson = ", theta[:eigs],";",
    end_davidson - start_davidson, "seconds")

# Begin Numpy diagonalization of A

start_numpy = time.time()

E,Vec = np.linalg.eig(A)
E = np.sort(E)

end_numpy = time.time()

# End of Numpy diagonalization. Print results.

print("numpy = ", E[:eigs],";",
     end_numpy - start_numpy, "seconds") 

davidson =  [1.00536285 1.98923559 3.01954143] ; 1.6152057647705078 seconds
numpy =  [1.00535621 1.98922871 3.01951537] ; 4.987552165985107 seconds


In [7]:
s.shape

(96, 96)